<a href="https://colab.research.google.com/github/joaugustc/analise_futebol_pyspark/blob/main/fraud_transaction_analysis_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Análise de perfil transacional
## tabelas utilizadas:


*   [fraud transactions dataset ](https://www.kaggle.com/datasets/dermisfit/fraud-transactions-dataset/data);
*   [bin list data](https://github.com/iannuttall/binlist-data/blob/e90a68a7baee2bfc84c9d07a71096ceac43e9e2d/binlist-data.csv).

## pacotes específicos:



*   geopy objetivando conseguir dados de geolocalização real com base na latitude e longitude do portador do cartão;
*   fuzzywuzzy para comparar similaridade entre nomenclaturas de localizações.






## 1. 📦 instalação de pacotes e importação dos dados

- instalação de pacotes necessários (`pandas-summary`, `geopy`, `fuzzywuzzy`)

In [ ]:
!pip install pandas-summary
!pip install geopy
!pip install fuzzywuzzy[speedup]

- importação dos arquivos

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# Importando os dados e capturando uma amostra dos registros de cada variável
import pandas as pd
df = pd.read_csv('fraud_transactions.csv')
df.sample(10)

## 2. 🧹 preparação e análise inicial dos dados




In [ ]:
# Limpeza de uma coluna do dataframe
df = df.drop(columns=['Unnamed: 0'])

In [ ]:
# Visualização de uma amostra do dataframe atualizado
df.sample(10)

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
147746,2020-08-11 14:46:15,3547560454237873,fraud_Adams-Barrows,health_fitness,18.05,Jerry,Robinson,M,212 Mary Stream,Arcadia,...,44.5232,-86.2061,680,"Designer, jewellery",1966-12-21,54fd8cb8bd9bcf22090a89ec21d385d5,1376232375,45.309026,-85.385879,0
4285,2020-06-22 15:13:22,577588686219,fraud_Thiel-Thiel,entertainment,105.36,James,Strickland,M,25454 Leonard Lake,Spring Church,...,40.6153,-79.4545,972,Public relations account executive,1997-10-23,a761d7fe417e8a90eb53e8706b982a3c,1371914002,39.875120,-79.505419,0
144246,2020-08-10 17:17:24,376944481517097,fraud_Beer-Jast,kids_pets,37.75,Tiffany,Stephens,F,1447 Ryan Lodge,Grand Bay,...,30.4983,-88.3282,15647,"Scientist, physiological",1957-12-26,fb1e5f46474bebadc342c00756f4005a,1376155044,30.652056,-89.274083,0
363850,2020-11-09 14:33:12,373213026644490,"fraud_Koss, Hansen and Lueilwitz",home,19.89,Bobby,Smith,M,3495 Williams Stream,San Diego,...,32.7185,-117.1593,1241364,Comptroller,1987-11-30,be36c16cc109da4f5b724783c1681309,1384007592,33.322463,-117.928668,0
179763,2020-08-23 16:49:04,503848303379,fraud_Kilback LLC,food_dining,40.22,Gregory,Wallace,M,27203 Darrell Landing,Saint James City,...,26.5290,-82.0916,3776,Sport and exercise psychologist,1976-09-12,9b117e429c4e00711999cc4d2096abef,1377276544,26.556142,-81.521469,0
466513,2020-12-12 15:26:42,377026671291680,fraud_McDermott-Rice,misc_pos,5.46,Jackie,Davis,F,1898 Parker Fork Apt. 057,Redford,...,37.3272,-91.0243,241,Investment analyst,1974-10-27,fff400424bf4ded8fabc78232db7591f,1386862002,37.004810,-90.470254,0
519432,2020-12-23 14:41:21,30561214688470,"fraud_Okuneva, Schneider and Rau",shopping_pos,45.30,Gina,Morrison,F,41851 Victor Drives Suite 219,Allentown,...,40.5891,-75.4645,166081,"Scientist, research (maths)",1998-10-01,aa38baeeb4cef8dfbc5136c655e95859,1387809681,40.770161,-75.551791,0
34652,2020-07-03 09:26:09,180046617132290,"fraud_Schoen, Kuphal and Nitzsche",grocery_pos,73.49,Erika,Mason,F,083 Karen Island Apt. 656,Goreville,...,37.5750,-88.9655,3119,"Teacher, secondary school",1942-04-17,d61cab55e8cf217038768ba4283b2dd6,1372843569,38.048973,-89.352288,0
306713,2020-10-15 09:57:12,4079773899158,fraud_Friesen Inc,shopping_pos,148.18,Eric,Preston,M,7020 Doyle Stream Apt. 951,Mesa,...,44.6255,-116.4493,129,Cartographer,1965-12-15,b6307426eebd6ce8e938fbecce27ee72,1381831032,44.156767,-115.749154,0
258555,2020-09-24 05:41:52,4610050989831291,fraud_Cormier LLC,shopping_net,102.07,James,Lynch,M,5155 Mary Common Apt. 927,Utica,...,41.4798,-79.9403,1102,Garment/textile technologist,1989-07-08,cf591ed68b00e83440f1f7704d9d1460,1380001312,40.578989,-80.690776,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555719 entries, 0 to 555718
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   trans_date_trans_time  555719 non-null  object 
 1   cc_num                 555719 non-null  int64  
 2   merchant               555719 non-null  object 
 3   category               555719 non-null  object 
 4   amt                    555719 non-null  float64
 5   first                  555719 non-null  object 
 6   last                   555719 non-null  object 
 7   gender                 555719 non-null  object 
 8   street                 555719 non-null  object 
 9   city                   555719 non-null  object 
 10  state                  555719 non-null  object 
 11  zip                    555719 non-null  int64  
 12  lat                    555719 non-null  float64
 13  long                   555719 non-null  float64
 14  city_pop               555719 non-nu

In [ ]:
# Alterando o formato da coluna "trans_date_trans_time" para datetime após observar o retorno de df.info()
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])

In [ ]:
# Capturando algumas informações estatísticas gerais de colunas numéricas específicas do dataframe
from pandas_summary import DataFrameSummary

cols_selecionadas = ['amt', 'city_pop']

df_selecionado = df[cols_selecionadas]

dfs = DataFrameSummary(df_selecionado)
summary = dfs.summary()

summary

,amt,city_pop
count,555719.0,555719.0
mean,69.39281,88221.887918
std,156.745941,300390.892061
min,1.0,23.0
25%,9.63,741.0
50%,47.29,2408.0
75%,83.01,19685.0
max,22768.11,2906700.0
counts,555719,555719
uniques,37256,835


## 4. 🧪 engenharia de variáveis

### 4.1 📍 variáveis de localização

In [ ]:
# Junção entre as variáveis city e state, seguidas por um complemento contendo o país
# Os retornos foram ajustados para texto em minúsculo para evitar informações duplicadas com formatações distintas
df['city_state'] = df['city'] + ', ' + df['state']
df['city_state_country'] = df['city_state'] + ', USA'
df['city_state_country'] = df['city_state_country'].str.lower()

In [ ]:
# Verificar 10 valores únicos da variável criada
print(df['city_state_country'].unique()[:10])

['columbia, sc, usa' 'altonah, ut, usa' 'bellmore, ny, usa'
 'titusville, fl, usa' 'falmouth, mi, usa' 'breesport, ny, usa'
 'carlotta, ca, usa' 'spencer, sd, usa' 'morrisdale, pa, usa'
 'prairie hill, tx, usa']


### 4.2 💳 variáveis do cartão

In [ ]:
# Criação de uma variável para utilização do BIN do cartão, dado referente ao banco emissor
df['bin'] = df['cc_num'].astype(str).str[:6]

In [ ]:
# Verificar 10 valores únicos da variável criada
print(df['bin'].unique()[:10])

['229116' '357303' '359821' '359191' '352682' '304076' '213180' '358928'
 '359635' '354689']


In [ ]:
# Lendo a base externa de bins
bin_data = pd.read_csv('binlist-data.csv')  # deve conter uma coluna com o bin e o banco

In [ ]:
# Transformando a variável bin em string e circustanciando à necessidade de ser composta por 6 dígitos para evitar dados inválidos
bin_data['bin'] = bin_data['bin'].astype(str)
bin_data = bin_data[bin_data['bin'].str.len() == 6]

In [ ]:
# Laço de repetição criado para verificar uma amostra aleatória de cada nova variável
for col in bin_data.columns:
    print(f'\n Coluna: {col}')
    print(bin_data[col].dropna().unique()[:10])


 Coluna: bin
['103003' '111100' '111102' '111103' '111301' '111904' '131000' '180000'
 '180001' '180002']

 Coluna: brand
['PRIVATE LABEL' 'JCB' 'ENROUTE' 'DISCOVER' 'AMERICAN EXPRESS'
 'DINERS CLUB' 'VISA' 'ATM' 'MAESTRO' 'DANKORT']

 Coluna: type
['DEBIT' 'CREDIT' 'CHARGE CARD' 'PREPAID']

 Coluna: category
['BUSINESS CARD' 'ATM ONLY' 'BUSINESS' 'CLASSIC' 'PREPAID'
 'CONSUMER PREMIUM CARD' 'GOLD' 'GOLD BUSINESS' 'PLATINUM' 'STANDARD']

 Coluna: issuer
['JCB CO., LTD.' 'AIR CANADA' 'DINERS CLUB INTERNATIONAL' 'ATOS'
 'ING BANK SLASKI, S.A.' 'BRANCH BANKING AND TRUST COMPANY'
 'BANK MILlanNIUM SPOLKA AKCYJNA' 'AMERICAN EXPRESS'
 'WESTPAC BANKING CORPORATION' 'HSBC BANK NEVADA, N.A.']

 Coluna: alpha_2
['US' 'JP' 'CA' 'GB' 'PL' 'HK' 'KR' 'CN' 'TH' 'EC']

 Coluna: alpha_3
['USA' 'JPN' 'CAN' 'GBR' 'POL' 'HKG' 'KOR' 'CHN' 'THA' 'ECU']

 Coluna: country
['United States' 'Japan' 'Canada' 'United Kingdom' 'Poland' 'Hong Kong'
 'SOUTH KOREA' 'China' 'Thailand' 'Ecuador']

 Coluna: latitude
[3

In [ ]:
# Agregando as novas informações através de merge entre os bins
df = df.merge(bin_data, how='left', left_on='bin', right_on='bin')

In [ ]:
# Visualização de uma amostra do dataframe atualizado
df.sample(10)

,trans_date_trans_time,cc_num,merchant,category_x,amt,first,last,gender,street,city,...,type,category_y,issuer,alpha_2,alpha_3,country,latitude,longitude,bank_phone,bank_url
367324,2020-11-10 15:37:19,3519232971341141,"fraud_Larkin, Stracke and Greenfelder",entertainment,1.71,Michael,Jones,M,754 Smith Isle,Amsterdam,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
419932,2020-12-01 16:19:12,4428780983793657331,fraud_Beer-Jast,kids_pets,93.73,Richard,Waters,M,7683 Natasha Way Apt. 945,Waukesha,...,CREDIT,NaN,BOEING EMPLOYEES C.U.,US,USA,United States,37.0902,-95.71290,800-233-2328,www.becu.org
295313,2020-10-10 20:14:12,675945690716,fraud_Koepp-Witting,grocery_pos,100.46,Ellen,Smith,F,285 George Lake,Beaver Falls,...,DEBIT,NaN,ALLOCATED TO SWITCH,GB,GBR,United Kingdom,55.3781,-3.43597,NaN,NaN
58984,2020-07-12 01:37:37,3559679414981506,fraud_Boyer PLC,misc_pos,13.47,Olivia,Stephenson,F,292 Cassidy Oval,Lowville,...,CREDIT,NaN,"JCB INTERNATIONAL CREDIT CARD CO., LTD.",US,USA,United States,37.0902,-95.71290,1-877-481-4711,www.jcbusa.com
157044,2020-08-15 16:27:26,373905417449658,fraud_Crona and Sons,kids_pets,210.72,Sarah,Bishop,F,554 Mcdonald Valley Apt. 539,Meridian,...,CREDIT,NaN,AMERICAN EXPRESS COMPANY,US,USA,United States,37.0902,-95.71290,NaN,NaN
20897,2020-06-28 17:35:30,6538441737335434,fraud_Corwin-Romaguera,travel,5.66,Gina,Grimes,F,444 Robert Mews,Clarks Mills,...,CREDIT,REWARDS CARD,BANK OF AMERICA,US,USA,United States,37.0902,-95.71290,800 732 9194,www.bankofamerica.com
113949,2020-07-31 04:29:43,3518669219150142,fraud_Mraz-Herzog,gas_transport,50.46,Dorothy,Thomas,F,537 Rice Square Suite 040,Milford,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
234871,2020-09-14 03:12:55,3540862542587229,fraud_Mante Group,misc_net,4.44,Nancy,Roach,F,376 Brown Courts,Yellowstone National Park,...,CREDIT,NaN,"JCB INTERNATIONAL CREDIT CARD CO., LTD.",US,USA,United States,37.0902,-95.71290,1-877-481-4711,www.jcbusa.com
509348,2020-12-21 16:55:39,4957924165498,fraud_Lubowitz-Walter,kids_pets,144.18,Alan,Parsons,M,0547 Russell Ford Suite 574,Kirk,...,CREDIT,NaN,NaN,US,USA,United States,37.0902,-95.71290,NaN,NaN
382089,2020-11-17 06:23:22,60427851591,fraud_Haley Group,misc_pos,29.62,Bradley,Martinez,M,3426 David Divide Suite 717,Burns Flat,...,DEBIT,NaN,GE CAPITAL CONSUMER CARD CO.,US,USA,United States,37.0902,-95.71290,NaN,NaN


In [ ]:
# Retirando colunas dispensáveis e renomeando outras variáveis
df = df.drop(columns=['alpha_2', 'alpha_3', 'latitude', 'longitude', 'bank_phone', 'bank_url'])
df = df.rename(columns={'country': 'country_card'})
df = df.rename(columns={'category_x': 'category_transaction'})
df = df.rename(columns={'category_y': 'category_card'})

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555719 entries, 0 to 555718
Data columns (total 30 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   trans_date_trans_time  555719 non-null  datetime64[ns]
 1   cc_num                 555719 non-null  int64         
 2   merchant               555719 non-null  object        
 3   category_transaction   555719 non-null  object        
 4   amt                    555719 non-null  float64       
 5   first                  555719 non-null  object        
 6   last                   555719 non-null  object        
 7   gender                 555719 non-null  object        
 8   street                 555719 non-null  object        
 9   city                   555719 non-null  object        
 10  state                  555719 non-null  object        
 11  zip                    555719 non-null  int64         
 12  lat                    555719 non-null  floa

### 4.3 💰 faixa de preço da transação

In [ ]:
# Criando uma coluna referente à faixa de preço
bins = [0, 25, 50, 100, 250, 500, 1000, df['amt'].max()]
labels = ['1. 0-25', '2. 26-50', '3. 51-100', '4. 101-250', '5. 251-500', '6. 501-1000', '7. 1000+']

df['price_range'] = pd.cut(df['amt'], bins=bins, labels=labels, include_lowest=True)

In [ ]:
# Verificar as 10 primeiras linhas da coluna criada
print(df['price_range'].head(10))

0       1. 0-25
1      2. 26-50
2      2. 26-50
3     3. 51-100
4       1. 0-25
5       1. 0-25
6    4. 101-250
7       1. 0-25
8       1. 0-25
9     3. 51-100
Name: price_range, dtype: category
Categories (7, object): ['1. 0-25' < '2. 26-50' < '3. 51-100' < '4. 101-250' < '5. 251-500' <
                         '6. 501-1000' < '7. 1000+']


### 4.4 🕒 turno da transação

In [ ]:
# Criando uma coluna referente ao turno de realização da transação

# Extraindo a hora da variável trans_date_trans_time
df['hora'] = df['trans_date_trans_time'].dt.hour

# Definindo o turno com base na hora
def classificar_turno(hora):
    if 0 <= hora < 6:
        return 'madrugada'
    elif 6 <= hora < 12:
        return 'manhã'
    elif 12 <= hora < 18:
        return 'tarde'
    else:
        return 'noite'

df['turno'] = df['hora'].apply(classificar_turno)

In [ ]:
# Verificar 10 registros aleatórios da coluna criada
print(df['turno'].sample(10, random_state=42))

119106        manhã
179292        tarde
540729        tarde
374360        manhã
314574    madrugada
377296        tarde
399943        noite
1481          noite
465665        manhã
430957        tarde
Name: turno, dtype: object


### 4.5 👤 faixa etária do portador

In [ ]:
# Criando uma coluna referente à faixa de idade do portador do cartão

from datetime import datetime

df['dob'] = pd.to_datetime(df['dob'], errors='coerce')

# calcular idade
ano_atual = datetime.now().year
df['idade'] = ano_atual - df['dob'].dt.year

# Definindo faixas com base na idade
bins = [0, 20, 34, 50, 65, 120]
labels = ['0-20', '21-34', '35-50', '51-65', '66+']

df['faixa_idade'] = pd.cut(df['idade'], bins=bins, labels=labels, right=True)

In [ ]:
# Verificar 10 registros aleatórios da coluna criada
print(df['faixa_idade'].sample(10, random_state=42))

119106    51-65
179292    35-50
540729      66+
374360    35-50
314574    51-65
377296    51-65
399943    21-34
1481      51-65
465665    35-50
430957    51-65
Name: faixa_idade, dtype: category
Categories (5, object): ['0-20' < '21-34' < '35-50' < '51-65' < '66+']


## 5. 🧪 construção da amostra

dado a grande volumetria de dados e pelo fato de estarmos trabalhando com uma tabela transacional fictícia foi extraída uma amostra para facilitar as operações posteriores, incluindo o uso do pacote geopy e a estruturação de dashboard gerencial



In [ ]:
# Definindo a amostra, para simular um cenário de alta em fraudes considerou-se
# um percentual de 10% de transações fraudulentas sobre o total de transações
n_total = 8000
p_fraude = 0.10
p_nao_fraude = 1 - p_fraude

total_fraude = df['is_fraud'].sum()
total_nao_fraude = (df['is_fraud'] == 0).sum()

n_fraude = min(int(n_total * p_fraude), total_fraude)
n_nao_fraude = min(int(n_total * p_nao_fraude), total_nao_fraude)

print(f"Total fraudes disponíveis: {total_fraude}")
print(f"Total não fraudes disponíveis: {total_nao_fraude}")
print(f"Fraudes para amostra: {n_fraude}")
print(f"Não fraudes para amostra: {n_nao_fraude}")

df_fraude = df[df['is_fraud'] == 1].sample(n=n_fraude, random_state=42)
df_nao_fraude = df[df['is_fraud'] == 0].sample(n=n_nao_fraude, random_state=42)

df_amostragem = pd.concat([df_fraude, df_nao_fraude]).sample(frac=1, random_state=42).reset_index(drop=True)

Total fraudes disponíveis: 2145
Total não fraudes disponíveis: 553574
Fraudes para amostra: 800
Não fraudes para amostra: 7200


## 6. 🌍 enriquecimento com geolocalização

In [ ]:
# Começando a utilização do geopy

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Inicializa o geolocalizador com timeout
geolocator = Nominatim(user_agent="geoapi", timeout=10)

geocode = RateLimiter(
    geolocator.reverse,
    min_delay_seconds=0.5,
    error_wait_seconds=5.0,
    max_retries=3,
    swallow_exceptions=True
)

# Cria dataframe com pares únicos de coordenadas
locais_unicos = df_amostragem[['lat', 'long']].drop_duplicates().reset_index(drop=True)

# Cria coluna 'coordenadas' no formato 'lat, long'
locais_unicos['coordenadas'] = locais_unicos.apply(lambda x: f"{x['lat']}, {x['long']}", axis=1)

# Aplica o geocode
locais_unicos['geolocalizacao'] = locais_unicos['coordenadas'].apply(geocode)

# Dicionário de siglas oficiais dos estados dos EUA
siglas_estados_eua = {
    'alabama': 'al', 'alaska': 'ak', 'arizona': 'az', 'arkansas': 'ar',
    'california': 'ca', 'colorado': 'co', 'connecticut': 'ct', 'delaware': 'de',
    'florida': 'fl', 'georgia': 'ga', 'hawaii': 'hi', 'idaho': 'id',
    'illinois': 'il', 'indiana': 'in', 'iowa': 'ia', 'kansas': 'ks',
    'kentucky': 'ky', 'louisiana': 'la', 'maine': 'me', 'maryland': 'md',
    'massachusetts': 'ma', 'michigan': 'mi', 'minnesota': 'mn', 'mississippi': 'ms',
    'missouri': 'mo', 'montana': 'mt', 'nebraska': 'ne', 'nevada': 'nv',
    'new hampshire': 'nh', 'new jersey': 'nj', 'new mexico': 'nm', 'new york': 'ny',
    'north carolina': 'nc', 'north dakota': 'nd', 'ohio': 'oh', 'oklahoma': 'ok',
    'oregon': 'or', 'pennsylvania': 'pa', 'rhode island': 'ri', 'south carolina': 'sc',
    'south dakota': 'sd', 'tennessee': 'tn', 'texas': 'tx', 'utah': 'ut',
    'vermont': 'vt', 'virginia': 'va', 'washington': 'wa', 'west virginia': 'wv',
    'wisconsin': 'wi', 'wyoming': 'wy'
}

# Função para formatar geolocalização e extrair cidade
def formatar_local(endereco):
    if endereco and endereco.raw.get('address'):
        address = endereco.raw['address']
        cidade = (address.get('city') or address.get('town') or address.get('village') or
                  address.get('hamlet') or address.get('county') or address.get('suburb') or '')
        estado_nome = (address.get('state') or '').lower()
        estado_sigla = siglas_estados_eua.get(estado_nome, estado_nome[:2])
        pais = address.get('country_code', '').lower()
        if cidade and estado_sigla and pais:
            return f"{cidade.lower()}, {estado_sigla.lower()}, {pais}"
    return None

# Função para extrair apenas a cidade
def extrair_cidade(endereco):
    if endereco and endereco.raw.get('address'):
        address = endereco.raw['address']
        cidade = (address.get('city') or address.get('town') or address.get('village') or
                  address.get('hamlet') or address.get('county') or address.get('suburb') or '')
        return cidade.lower()
    return None

# Aplicando as funções
locais_unicos['geolocalizacao_real'] = locais_unicos['geolocalizacao'].apply(formatar_local)
locais_unicos['cidade_geolocalizada'] = locais_unicos['geolocalizacao'].apply(extrair_cidade)

In [ ]:
# Verificando as novas variáveis
print(locais_unicos[['lat', 'long', 'geolocalizacao_real', 'cidade_geolocalizada']].head(10))

       lat      long                     geolocalizacao_real  \
0  42.2470  -73.7552                  city of hudson, ny, us   
1  38.4121  -75.2811                worcester county, md, us   
2  33.2887 -111.0985                        superior, az, us   
3  43.1128  -77.4906  town/village of east rochester, ny, us   
4  38.2507  -85.7476                      louisville, ky, us   
5  37.9943  -88.9417                       west city, il, us   
6  38.9999 -109.6150                    grand county, ut, us   
7  45.3637  -84.4695                cheboygan county, mi, us   
8  41.2639  -80.8164                 warren township, oh, us   
9  38.2911  -92.7059                   morgan county, mo, us   

             cidade_geolocalizada  
0                  city of hudson  
1                worcester county  
2                        superior  
3  town/village of east rochester  
4                      louisville  
5                       west city  
6                    grand county  
7      

In [ ]:
# Merge pela lat e long para adicionar a coluna geolocalizacao_resumida no df original
df_amostragem = df_amostragem.merge(
    locais_unicos[['lat', 'long', 'geolocalizacao_real', 'cidade_geolocalizada']],
    on=['lat', 'long'],
    how='left'
)

In [ ]:
# Criando uma variável binária para expor divergências entre a geolocalizacao real e a informação de localização preenchida pelo portador do cartão
from fuzzywuzzy import fuzz

def compara_locais(a, b, limiar=50):
    return int(fuzz.ratio(a, b) < limiar)

df_amostragem['divergencia_localizacao'] = df_amostragem.apply(
    lambda row: compara_locais(row['city'], row['cidade_geolocalizada']), axis=1
)

In [ ]:
# Verificando alguns registros
print(df_amostragem[['city', 'cidade_geolocalizada', 'divergencia_localizacao']].sample(10))

                     city cidade_geolocalizada  divergencia_localizacao
4837              Oolitic              oolitic                        0
7715           Stillwater           stillwater                        0
2093  Shenandoah Junction     jefferson county                        1
2203   Mount Saint Joseph   mount saint joseph                        0
4699          San Antonio          san antonio                        0
2594             Reynolds   grand forks county                        1
3928               Thrall    williamson county                        1
3426           Tower Hill           tower hill                        0
4483          Murrayville        morgan county                        1
6942                Halma                halma                        0


In [ ]:
# Visualização de uma amostra do dataframe final
df_amostragem.sample(10)

,trans_date_trans_time,cc_num,merchant,category_transaction,amt,first,last,gender,street,city,...,issuer,country_card,price_range,hora,turno,idade,faixa_idade,geolocalizacao_real,cidade_geolocalizada,divergencia_localizacao
5998,2020-12-03 03:05:44,2288813824604479,"fraud_Connelly, Reichert and Fritsch",gas_transport,74.15,Barbara,Norman,F,6278 Stephanie Unions,New York City,...,NaN,NaN,3. 51-100,3,madrugada,44,35-50,"city of new york, ny, us",city of new york,1
4512,2020-12-28 09:56:53,4488258847705225890,fraud_Kuvalis Ltd,gas_transport,66.81,Jessica,Shaw,F,5745 Blair Locks,West Columbia,...,CITY COUNTY CREDIT UNION OF FORT LAUDERDALE,United States,3. 51-100,9,manhã,96,66+,"springdale, sc, us",springdale,1
1849,2020-09-22 07:31:51,3519607465576254,"fraud_Huel, Hammes and Witting",grocery_pos,128.08,Audrey,Gonzalez,F,34180 Lopez Plaza,Fields Landing,...,NaN,NaN,4. 101-250,7,manhã,96,66+,"fields landing, ca, us",fields landing,0
3244,2020-10-06 22:40:28,581508178315,"fraud_Moore, Dibbert and Koepp",misc_net,798.61,Michael,Vargas,M,933 Martin Rapids,Desdemona,...,NaN,United States,6. 501-1000,22,noite,47,35-50,"eastland county, tx, us",eastland county,1
1873,2020-07-03 22:28:26,213178765398315,fraud_Cummerata-Hilpert,entertainment,10.28,Alan,Colon,M,748 Bryan Fields Suite 935,Riverton,...,"JCB CO., LTD.",Japan,1. 0-25,22,noite,32,21-34,"fremont county, wy, us",fremont county,1
1564,2020-12-08 18:34:30,6011652924285713,fraud_Nader-Maggio,personal_care,138.48,Kathryn,Smith,F,19838 Tonya Prairie Apt. 947,Rocky Mount,...,NaN,NaN,4. 101-250,18,noite,37,35-50,"morgan county, mo, us",morgan county,0
2747,2020-12-07 10:05:36,6526955903501879,"fraud_Goyette, Howell and Collier",shopping_pos,33.03,Jennifer,Scott,F,7351 Cindy Well Suite 099,Naples,...,BANK OF AMERICA,United States,2. 26-50,10,manhã,75,66+,"collier county, fl, us",collier county,1
2799,2020-10-25 03:34:13,3576431665303017,fraud_Huel Ltd,misc_net,152.99,Jessica,Ward,F,72269 Elizabeth Field Apt. 132,Phoenix,...,"JCB INTERNATIONAL CREDIT CARD CO., LTD.",United States,4. 101-250,3,madrugada,44,35-50,"phoenix, az, us",phoenix,0
4486,2020-06-21 19:23:19,3586008444788268,fraud_Terry-Huel,shopping_net,480.56,Crystal,Fuller,F,000 Jennifer Mills,Issaquah,...,"JCB INTERNATIONAL CREDIT CARD CO., LTD.",United States,5. 251-500,19,noite,41,35-50,"king county, wa, us",king county,1
1275,2020-10-14 13:50:45,373043435942726,fraud_Pfeffer and Sons,shopping_pos,132.58,Vincent,Richards,M,2064 Tyler Fork,Beasley,...,AMERICAN EXPRESS COMPANY,United States,4. 101-250,13,tarde,44,35-50,"fort bend county, tx, us",fort bend county,1


## 8. 💾 exportação do dataset final

In [ ]:
df_amostragem.to_excel('amostra.xlsx', index=False)

In [ ]:
from google.colab import files
files.download('amostra.xlsx')